In [64]:
import pandas as pd

import numpy as np
import json
import geog
import shapely.geometry
import geopandas as gpd
import folium
from shapely import wkt, geometry
import json
from pprint import pprint
from folium import plugins

In [65]:
data_clean = pd.read_csv('listings_cleaned.csv', index_col = 'id')
data = pd.read_csv('listings.csv', index_col = 'id')

C:\Users\sgevo\Anaconda3\envs\ox\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [66]:
cta_json = gpd.read_file('cta_entrances.geojson')

In [67]:
cta_json

,name,agency,line,geometry
0,18th,CTA,Pink Line,POINT (-87.66914 41.85785)
1,35th/Archer,CTA,Orange Line,POINT (-87.68063 41.82927)
2,95th-Dan Ryan,CTA,Red Line,POINT (-87.62441 41.72273)
3,Adams/Wabash,CTA,"Brown, Purple, Orange, Pink, Green Lines",POINT (-87.62600 41.87971)
4,Addison,CTA,Blue Line,POINT (-87.71841 41.94660)
...,...,...,...,...
376,Washington/Wells,CTA,"Brown, Purple, Orange, Pink, Green Lines",POINT (-87.63378 41.88299)
377,Washington/Wabash,CTA,"Brown, Purple, Orange, Pink, Green Lines",POINT (-87.62609 41.88231)
378,Washington/Wabash,CTA,"Brown, Purple, Orange, Pink, Green Lines",POINT (-87.62626 41.88232)
379,Washington/Wabash,CTA,"Brown, Purple, Orange, Pink, Green Lines",POINT (-87.62609 41.88297)


In [68]:
import random 
random_row = random.randint(0,len(data))
random_row 

5853

In [69]:
base_long = data.longitude.iloc[random_row]
base_lat = data.latitude.iloc[random_row]

In [70]:
p = shapely.geometry.Point([base_long,base_lat])
n_points = 100
d = 1000  # meters
angles = np.linspace(0, 360, n_points)
polygon = geog.propagate(p, angles, d)
wkt_str = shapely.geometry.Polygon(polygon).wkt
#wkt_str = json.dumps(shapely.geometry.mapping(shapely.geometry.Polygon(polygon)))
#print(shapely.geometry.mapping(shapely.geometry.Polygon(polygon)))
aoi_geom = wkt.loads(wkt_str)
poly = shapely.geometry.Polygon(polygon)

In [71]:
aoi_coords = list(aoi_geom.exterior.coords) # get coords from exterior ring
aoi_coords = [(y,x) for x,y in aoi_coords] # swap (x,y) to (y,x). Really leaflet?!
aoi_centroid = aoi_geom.centroid # Kreuzberg center for map center

In [72]:
m = folium.Map(tiles='Cartodb Positron',location=(aoi_centroid.y, aoi_centroid.x), zoom_start=15)

In [73]:
m

In [74]:
icon = folium.map.Icon(color='darkblue',
                        icon_color='#ade8f4',
                        icon='bed', # fetches font-awesome.io symbols
                        prefix='fa')
popup = "<strong>{0}</strong><br>Accommodates: {1}<br>Bathrooms: {2}<br>Price: {3}<br>Link: {4}".format(data.loc[data.iloc[random_row].name]['name'],data.loc[data.iloc[random_row].name]['accommodates'], data.loc[data.iloc[random_row].name]['bathrooms'],data.loc[data.iloc[random_row].name]['price'],data.loc[data.iloc[random_row].name]['listing_url'])
folium.map.Marker([base_lat,base_long], icon=icon, popup = popup).add_to(m)
m

In [75]:
folium.vector_layers.Polygon(aoi_coords,
                                     color='#FCA311',
                                     fill_color='#DEB841',
                                    fill_opacity=0.05,
                                     weight=3).add_to(m)
m

In [76]:
from openrouteservice import client, places
api_key='5b3ce3597851110001cf6248f4a7fee5db334593bc67406f86071778'
clnt = client.Client(key=api_key)
routes = {}
times = []
list_loc = [[base_long,base_lat]]
n = 0 
for name, point, line, x in zip(cta_json['name'], cta_json['geometry'], cta_json['line'], range(len(cta_json))):
    if point.within(poly):
        folium.Marker([point.y, point.x], popup= "<strong>{0}</strong><br>{1}<br>".format(name, line),
                      icon = folium.Icon(color='green',
                                icon_color='#FFFFFF',
                                icon='subway', # fetches font-awesome.io symbols
                                prefix='fa')).add_to(m)
        cta_loc = [[point.x,point.y]] 
        route_coords =  list_loc + cta_loc  
        request = {'coordinates': route_coords,
           'profile': 'foot-walking',
           'geometry': 'true',
           'format_out': 'geojson'}
        route = clnt.directions(**request)
        routes[n] = route
        times.append(route['features'][0]['properties']['summary']['duration'])
        n += 1
    else:
        folium.CircleMarker(
            [point.y, point.x],
            radius=3,
            popup = "<strong>{0}</strong><br>{1}<br>".format(name, line),
            color='red',
            fill=True,
            fill_opacity=0.7
            ).add_to(m)


In [77]:
def style_function(color):
    return lambda feature: dict(color=color,
                              weight=7,
                              opacity=1)

In [78]:
tooltip = folium.features.GeoJsonTooltip(
    fields=(["summary"]),
    aliases=["Route Distance in Meters and Duration of the route (yet in seconds): "],
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

In [79]:
def route_plot(routes_matrix):
    fastest = None
    if not routes:
        print('No CTAs in the area')
    else:
        for x in routes:
            if routes[x]['features'][0]['properties']['summary']['duration'] == min(times):
                fastest = x
            else:
                folium.features.GeoJson(data=routes[x],
                                name='Route' + str(x),
                                style_function= style_function('#DEB841'),
                                tooltip = folium.map.Tooltip(("Route Distance: {0} meters<br>Duration of the route: {1:.1f} minutes<br>".format(routes[x]['features'][0]['properties']['summary']['distance'], routes[x]['features'][0]['properties']['summary']['duration']/60))),
                               overlay=True).add_to(m)
        if fastest is not None:
            folium.features.GeoJson(data=routes[fastest],
                                name='Fastest Route',
                                style_function= style_function('#0ead69'),
                                tooltip = folium.map.Tooltip(("<strong>Route Distance: {0} meters</strong><br><strong>Duration of the route: {1:.1f} minutes <strong><br>".format(routes[fastest]['features'][0]['properties']['summary']['distance'], routes[fastest]['features'][0]['properties']['summary']['duration']/60))),
                                popup = folium.map.Popup("<strong>FASTEST ROUTE</strong>"),
                               overlay=True).add_to(m)
    folium.map.LayerControl().add_to(m)
    return m
            

In [80]:
route_plot(routes)